In [1]:
import opal
import kglab
import rdflib
import z3
import dateutil.parser
import re
import numpy as np
import pandas as pd
import os
import sexpdata

In [2]:
from opal.logic.z3.z3_literal import Z3Literal
from opal.data.event_data import EventData
from opal.data.mapper import LogMapper
from opal.data.processor import LogProcessor

import opal.logic.z3.z3_literal as z3_literal
from opal.logic.z3.z3_ontology import Z3Ontology as Ontology
from opal.logic.z3.z3_reasoner import Z3Reasoner as Reasoner

from rdflib import Graph

from z3 import *
z3.set_param(proof=True)

Adding axiom: PSL Core Ontology_ont_t_0_type_disjointness
Adding axiom: PSL Core Ontology_ont_t_1_begin_unique
Adding axiom: PSL Core Ontology_ont_t_2_ends_unique
Adding axiom: PSL Core Ontology_ont_t_3_occurrence_bounds
Adding axiom: PSL Core Ontology_ont_t_4_occurrence_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_5_unique_activity_occurrence
Adding axiom: PSL Core Ontology_ont_t_6_occurrence_has_activity
Adding axiom: PSL Core Ontology_ont_t_7_participation_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_8_participation_temporal_constraint
Adding axiom: PSL Core Ontology_ont_t_9_object_temporal_existence
Adding axiom: PSL Core Ontology_ont_t_10_occurrence_temporal_extent
Adding axiom: PSL Core Ontology_map_t_0_occurrence_start_end_event_mapping
Adding axiom: PSL Core Ontology_map_t_1_occurrence_participation_mapping
Adding axiom: PSL Occtree Ontology_ont_t_0_arboreal_occurrence_restriction
Adding axiom: PSL Occtree Ontology_ont_t_1_earlier_requires_arboreal
Adding ax

In [3]:
data_dir = os.path.abspath('../data/')
ttl_data_file = data_dir + '/P1_log_instances.ttl'
sample_log_file = data_dir + '/sample_log.csv'
bpic2017_file = data_dir + '/BPI Challenge 2017.xes.gz'

ontology_dir = os.path.abspath('../../opal/logic/z3/ontologies/')
psl_core_file = ontology_dir + '/PSL/PSL_core.smt2'
psl_core_mapping_file = ontology_dir + '/PSL/PSL_core_mapping.smt2'

## Mapping simple XES data into logical facts

In [4]:
# load the log
col_dict = {'caseID' : 'caseID', 'activityID' : 'activityID', 'timestamp' : 'timestamp', 'eventID' : None, 'transition' : 'transition'}
processor = LogProcessor(col_dict=col_dict, log_path=sample_log_file, batch_size=1)
event_data = processor.load_log()

In [5]:
event_data.data

,caseID,activityID,eventID,timestamp,resourceID,transition,processID,batch
0,C_case_0,A_activity_A,E_0,2016-01-01 09:00:00+00:00,user_1,T_start,P1,1
1,C_case_0,A_activity_A,E_1,2016-01-01 09:15:00+00:00,user_1,T_complete,P1,1
2,C_case_0,A_activity_C,E_2,2016-01-01 09:35:00+00:00,user_1,T_complete,P1,1
3,C_case_1,A_activity_A,E_3,2016-01-02 09:00:00+00:00,user_1,T_complete,P1,2
4,C_case_1,A_activity_B,E_4,2016-01-02 09:00:00+00:00,user_0,T_complete,P1,2
5,C_case_1,A_activity_C,E_5,2016-01-03 09:00:00+00:00,user_0,T_complete,P1,2
6,C_case_2,A_activity_A,E_6,2016-01-01 09:00:00+00:00,user_1,T_complete,P1,3
7,C_case_2,A_activity_B,E_7,2016-01-01 09:10:00+00:00,user_1,T_complete,P1,3
8,C_case_2,A_activity_C,E_8,2016-01-01 09:15:00+00:00,user_2,T_complete,P1,3
9,C_case_2,A_activity_D,E_9,2017-01-06 10:35:00+00:00,user_1,T_complete,P1,3


In [6]:
event_data.metadata

{'log_paths': ['/tmp/tmpqw_bmh90.csv',
  '/tmp/tmp6j_smq5q.csv',
  '/tmp/tmppbc_ww6s.csv'],
 'process_name': 'P1',
 'batches': 3}

In [7]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/3 [00:00<?, ?it/s]2025-09-23 18:27:11,127 | INFO: Translating YARRRML mapping to [R2]RML
2025-09-23 18:27:11,128 | INFO: RML content is created!
2025-09-23 18:27:11,136 | INFO: Mapping has been syntactically validated.
2025-09-23 18:27:11,137 | INFO: Translation has finished successfully.
2025-09-23 18:27:11,144 | INFO: Translating YARRRML mapping to [R2]RML
2025-09-23 18:27:11,145 | INFO: RML content is created!
2025-09-23 18:27:11,150 | INFO: Mapping has been syntactically validated.
2025-09-23 18:27:11,151 | INFO: Translation has finished successfully.
2025-09-23 18:27:11,158 | INFO: Translating YARRRML mapping to [R2]RML
2025-09-23 18:27:11,159 | INFO: RML content is created!
2025-09-23 18:27:11,166 | INFO: Mapping has been syntactically validated.
2025-09-23 18:27:11,167 | INFO: Translation has finished successfully.
100%|██████████| 3/3 [00:00<00:00, 62.71it/s]

Mappings built.
Generating knowledge graph...



INFO | 2025-09-23 18:27:11,689 | 11 mapping rules retrieved.
INFO | 2025-09-23 18:27:11,701 | Mapping partition with 11 groups generated.
INFO | 2025-09-23 18:27:11,702 | Maximum number of rules within mapping group: 1.
INFO | 2025-09-23 18:27:11,704 | Mappings processed in 0.532 seconds.
INFO | 2025-09-23 18:27:12,112 | Number of triples generated in total: 25.


Knowledge graph generated.
Knowledge graph 1/3 generated.
Generating knowledge graph...


INFO | 2025-09-23 18:27:12,604 | 11 mapping rules retrieved.
INFO | 2025-09-23 18:27:12,617 | Mapping partition with 11 groups generated.
INFO | 2025-09-23 18:27:12,618 | Maximum number of rules within mapping group: 1.
INFO | 2025-09-23 18:27:12,619 | Mappings processed in 0.497 seconds.
INFO | 2025-09-23 18:27:12,975 | Number of triples generated in total: 26.


Knowledge graph generated.
Knowledge graph 2/3 generated.
Generating knowledge graph...


INFO | 2025-09-23 18:27:13,458 | 11 mapping rules retrieved.
INFO | 2025-09-23 18:27:13,470 | Mapping partition with 11 groups generated.
INFO | 2025-09-23 18:27:13,471 | Maximum number of rules within mapping group: 1.
INFO | 2025-09-23 18:27:13,472 | Mappings processed in 0.490 seconds.
INFO | 2025-09-23 18:27:13,810 | Number of triples generated in total: 33.


Knowledge graph generated.
Knowledge graph 3/3 generated.
Z3 literal set generated (1/3)
Z3 literal set generated (2/3)
Z3 literal set generated (3/3)


In [8]:
from opal.logic.z3.z3_handler import PSL_CORE_MAPPING_ENV
for literal in z3_data:
    expr = literal.z3_expr
    print(expr)
    
# get a random expression to check if contexts remain the same with additional ontologies loaded
random_idx = np.random.randint(len(z3_data))
random_expr = z3_data[random_idx].z3_expr
print(f'Matching contexts: {random_expr.ctx == PSL_CORE_MAPPING_ENV["ctx"]}')

hasRecordedTime(E_2, 1451640900)
Transition(T_start)
Case(C_case_0)
Event(E_1)
Event(E_0)
hasProcess(C_case_0, P1)
Resource(user_1)
hasCase(E_1, C_case_0)
hasRecordedTime(E_1, 1451639700)
hasActivity(E_2, A_activity_C)
hasTransition(E_1, T_complete)
hasTransition(E_0, T_start)
hasActivity(E_1, A_activity_A)
Event(E_2)
hasResource(E_1, user_1)
hasResource(E_0, user_1)
hasActivity(E_0, A_activity_A)
hasCase(E_2, C_case_0)
hasTransition(E_2, T_complete)
hasCase(E_0, C_case_0)
Transition(T_complete)
hasRecordedTime(E_0, 1451638800)
hasResource(E_2, user_1)
Activity(A_activity_A)
Activity(A_activity_C)
hasResource(E_4, user_0)
hasProcess(C_case_1, P1)
hasTransition(E_4, T_complete)
hasTransition(E_5, T_complete)
hasRecordedTime(E_5, 1451811600)
Activity(A_activity_A)
Resource(user_1)
hasActivity(E_4, A_activity_B)
hasCase(E_3, C_case_1)
Event(E_5)
hasRecordedTime(E_4, 1451725200)
Event(E_3)
hasResource(E_5, user_0)
hasRecordedTime(E_3, 1451725200)
Case(C_case_1)
hasResource(E_3, user_1)
has

In [9]:
PSL_CORE_MAPPING_ENV['functions'] 

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of,
 'arboreal': arboreal,
 'earlier': earlier,
 'earlierEq': earlierEq,
 'initial': initial,
 'generator': generator,
 'legal': legal,
 'precedes': precedes,
 'poss': poss,
 'successor': successor,
 'hasProcess': hasProcess,
 'hasTransition': hasTransition}

### Reasoning Capabilities

In [10]:
# load the ontology (together with the mapping)
psl_core = Ontology.from_smt2(psl_core_file, psl_core_mapping_file)

Adding axiom: None_ont_t_0_type_disjointness
Adding axiom: None_ont_t_1_begin_unique
Adding axiom: None_ont_t_2_ends_unique
Adding axiom: None_ont_t_3_occurrence_bounds
Adding axiom: None_ont_t_4_occurrence_sort_constraints
Adding axiom: None_ont_t_5_unique_activity_occurrence
Adding axiom: None_ont_t_6_occurrence_has_activity
Adding axiom: None_ont_t_7_participation_sort_constraints
Adding axiom: None_ont_t_8_participation_temporal_constraint
Adding axiom: None_ont_t_9_object_temporal_existence
Adding axiom: None_ont_t_10_occurrence_temporal_extent
Adding axiom: None_map_t_0_occurrence_start_end_event_mapping
Adding axiom: None_map_t_1_occurrence_participation_mapping


In [12]:
psl_core.axioms

[{'name': 'None_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'None_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'None_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :description "End points are unique."))'},

In [13]:
psl_core.literals

[]

In [14]:
z3_facts = []
for i, fact in enumerate(z3_data):
    prefix = f'mapped_a_{i}_'
    expr = fact
    name = prefix + f'{expr}'
    z3_facts.append({name : expr})

In [15]:
from opal.logic.z3.z3_ontology import PSL_CORE_Z3, PSL_OCCTREE_Z3
print(PSL_CORE_Z3.name)
print(PSL_OCCTREE_Z3.name)

PSL Core Ontology
PSL Occtree Ontology


In [20]:
PSL_OCCTREE_Z3.axioms

[{'name': 'PSL Core Ontology_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'PSL Core Ontology_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'PSL Core Ontology_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :de

In [21]:
r = Reasoner()
r.load_ontology(PSL_OCCTREE_Z3)
r.load_facts(z3_facts)

In [22]:
s = r.check()

In [23]:
# Since the data is consistent with the ontology, we have a satisfying interpretation and can extract a model
print(s.model())

[PSL Occtree Ontology_ont_t_3_earlier_transitive = True,
 PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 mapped_a_7_hasCase(E_1, C_case_0) = True,
 mapped_a_47_Event(E_4) = True,
 mapped_a_59_Resource(user_1) = True,
 PSL Occtree Ontology_ont_t_1_earlier_requires_arboreal = True,
 A_activity_A = Ind!val!9,
 mapped_a_78_Event(E_6) = True,
 E_8 = Ind!val!17,
 E_0 = Ind!val!4,
 mapped_a_26_hasProcess(C_case_1, P1) = True,
 mapped_a_48_hasCase(E_4, C_case_1) = True,
 mapped_a_49_hasActivity(E_3, A_activity_A) = True,
 mapped_a_58_hasCase(E_6, C_case_2) = True,
 mapped_a_21_hasRecordedTime(E_0, 1451638800) = True,
 mapped_a_15_hasResource(E_0, user_1) = True,
 mapped_a_10_hasTransition(E_1, T_complete) = True,
 mapped_a_45_hasTransition(E_3, T_complete) = True,
 mapped_a_55_hasTransition(E_9, T_complete) = True,
 mapped_a_73_hasRecordedTime(E_6, 1451638800) = True,
 mapped_a_35_hasRecordedTime(E_4, 1451725200) = True,
 mapped_a_63_hasRecordedTime(E_9, 1483698900) = True,
 E_7 = 

In [17]:
from opal.logic.z3.z3_handler import REFERENCE_TAXONOMY_ENV
REFERENCE_TAXONOMY_ENV['constants']

{'T_start': T_start,
 'T_complete': T_complete,
 'E_1': E_1,
 'A_activity_A': A_activity_A,
 'E_2': E_2,
 'C_case_0': C_case_0,
 'user_1': user_1,
 'E_0': E_0,
 'P1': P1,
 'A_activity_C': A_activity_C,
 'E_3': E_3,
 'E_4': E_4,
 'E_5': E_5,
 'A_activity_B': A_activity_B,
 'C_case_1': C_case_1,
 'user_0': user_0,
 'E_9': E_9,
 'E_7': E_7,
 'C_case_2': C_case_2,
 'E_6': E_6,
 'user_2': user_2,
 'E_8': E_8,
 'A_activity_D': A_activity_D}

In [18]:
r.functions

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of,
 'hasTransition': hasTransition,
 'hasProcess': hasProcess,
 'arboreal': arboreal,
 'earlier': earlier,
 'earlierEq': earlierEq,
 'initial': initial,
 'generator': generator,
 'legal': legal,
 'precedes': precedes,
 'poss': poss,
 'successor': successor}

In [18]:
psl_core.env

{'ctx': <z3.z3.Context at 0x7d5a0951e7c0>,
 'sorts': {'Ind': Ind},
 'functions': {'Event': Event,
  'Transition': Transition,
  'Activity': Activity,
  'Case': Case,
  'Resource': Resource,
  'hasResource': hasResource,
  'hasRecordedTime': hasRecordedTime,
  'hasLifecycleTransition': hasLifecycleTransition,
  'hasCase': hasCase,
  'hasActivity': hasActivity,
  'activity': activity,
  'activity_occurrence': activity_occurrence,
  'object_': object_,
  'timepoint': timepoint,
  'occurrence_of': occurrence_of,
  'participates_in': participates_in,
  'exists_at': exists_at,
  'is_occurring_at': is_occurring_at,
  'begin_of': begin_of,
  'end_of': end_of,
  'hasTransition': hasTransition,
  'hasProcess': hasProcess,
  'transition': transition},
 'constants': {'T_start': T_start,
  'T_complete': T_complete,
  'E_2': E_2,
  'E_0': E_0,
  'user_1': user_1,
  'E_1': E_1,
  'A_activity_A': A_activity_A,
  'A_activity_C': A_activity_C,
  'C_case_0': C_case_0,
  'P1': P1,
  'E_4': E_4,
  'E_5': E

### Pattern Usage

# Mapping BPIC 2017 data

In [12]:
processor = LogProcessor(log_path=bpic2017_file, batch_size=100)
event_data = processor.load_log(downsample_rate=0.05)

/home/vscode/.local/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:57<00:00, 546.80it/s]
/workspaces/ProMean Lib/opal/data/processor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['batch'] = (df.groupby('caseID').ngroup() // self.batch_size) + 1


In [13]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/16 [00:00<?, ?it/s]2025-08-11 14:02:34,488 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,488 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,490 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,490 | RML content is created!
2025-08-11 14:02:34,496 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,496 | Mapping has been syntactically validated.
2025-08-11 14:02:34,498 | INFO: Translation has finished successfully.
INFO | 2025-08-11 14:02:34,498 | Translation has finished successfully.
2025-08-11 14:02:34,508 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,508 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,509 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,509 | RML content is created!
2025-08-11 14:02:34,516 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,516 | Mapping has been syntactically validated.
2025-08-1

Mappings built.
Generating knowledge graph...


INFO | 2025-08-11 14:02:35,287 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:02:35,297 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:02:35,299 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:02:35,299 | Mappings processed in 0.519 seconds.
INFO | 2025-08-11 14:02:36,870 | Number of triples generated in total: 23614.


Knowledge graph generated.
Knowledge graph 1/16 generated.
Generating knowledge graph...


...

Knowledge graph generated.
Knowledge graph 14/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:11,740 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:11,754 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:11,755 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:11,756 | Mappings processed in 0.540 seconds.
INFO | 2025-08-11 14:03:13,130 | Number of triples generated in total: 22243.


Knowledge graph generated.
Knowledge graph 15/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:15,046 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:15,057 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:15,058 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:15,059 | Mappings processed in 0.538 seconds.
INFO | 2025-08-11 14:03:16,380 | Number of triples generated in total: 17015.


Knowledge graph generated.
Knowledge graph 16/16 generated.
Z3 literal set generated (1/16)
Z3 literal set generated (2/16)
Z3 literal set generated (3/16)
Z3 literal set generated (4/16)
Z3 literal set generated (5/16)
Z3 literal set generated (6/16)
Z3 literal set generated (7/16)
Z3 literal set generated (8/16)
Z3 literal set generated (9/16)
Z3 literal set generated (10/16)
Z3 literal set generated (11/16)
Z3 literal set generated (12/16)
Z3 literal set generated (13/16)
Z3 literal set generated (14/16)
Z3 literal set generated (15/16)
Z3 literal set generated (16/16)


In [14]:
len(z3_data)

359577

In [ ]:
len(mapper.data.data)

59801

In [16]:
str(z3_data[5008])

'hasResource(E_1174728, R_User_29)'

### Creating a simple ontology and performing reasoning
Some code here is taken from the opal library for testing purposes to demonstrate how reasoning works. The actual opal library usage will be much simpler

In [ ]:
# Define a fixed universe of constants and predicate symbols
x = Const('x', z3_literal.Ind)

# Mapping of logical operators
logical_ops = {
    'Or': z3.Or,
    'And': z3.And,
    'Not': z3.Not,
    'Implies': z3.Implies,
    'ForAll': z3.ForAll,
    'Exists': z3.Exists
}

predicate_map = SIGNATURE_REGISTER

# Recursive parser
def parse_expr(expr_str, var_ctx={}):
    expr_str = expr_str.strip()
    if expr_str.startswith('ForAll') or expr_str.startswith('Exists'):
        quantifier, inner = expr_str.split('(', 1)
        args = inner.rsplit(')', 1)[0]
        var_part, body_part = args.split(',', 1)
        var_name = var_part.strip()
        var = var_ctx.get(var_name, z3.Const(var_name, z3_literal.Ind))
        return logical_ops[quantifier](var, parse_expr(body_part.strip(), {**var_ctx, var_name: var}))
    elif '(' in expr_str:
        func_name, inner = expr_str.split('(', 1)
        inner_args = [parse_expr(arg.strip(), var_ctx) for arg in split_args(inner.rsplit(')', 1)[0])]
        if func_name in logical_ops:
            return logical_ops[func_name](*inner_args)
        elif func_name in predicate_map:
            return predicate_map[func_name](*inner_args)
        else:
            # otherwise, create a new z3 function symbol
            func = Function(func_name, *([z3_literal.Ind] * len(inner_args)), BoolSort())
            predicate_map[func_name] = func
            return predicate_map[func_name](*inner_args)
    else:
        # It's a variable name
        return var_ctx.get(expr_str, z3.Const(expr_str, z3.IntSort()))

# Helper to split arguments in a safe way
def split_args(s):
    args, depth, current = [], 0, ''
    for c in s:
        if c == ',' and depth == 0:
            args.append(current)
            current = ''
        else:
            if c == '(':
                depth += 1
            elif c == ')':
                depth -= 1
            current += c
    if current:
        args.append(current)
    return args

In [ ]:
axiom_str = 'ForAll(x, Not(ActivityOccurrence(x)))'
test_axiom = parse_expr(axiom_str)
print(test_axiom)

ForAll(x, Not(ActivityOccurrence(x)))


In [ ]:
# try the solver on basic axioms
z3.set_param(proof=True)
s = Solver()
s.set(unsat_core=True)
s.set(completion=True)
for i,literal in enumerate(z3_data):
  s.assert_and_track(literal.z3_expr, 'ind_'+str(i))
s.assert_and_track(test_axiom, 'axiom')

In [ ]:
# model generation / satisfiability check
if s.check() == sat:
    m = s.model()
    print("Theory is satisfiable -- model generated.")
    print(m)
else:
    print("Theory is unsatisfiable -- no model generated.")
    #pr = s.proof()
    #pr_sexpr = pr.sexpr()
    core = s.unsat_core()
    print(core)

Model is consistent.
[ind_231450 = True,
 ind_266179 = True,
 ind_262416 = True,
 ind_249560 = True,
 ind_136661 = True,
 ind_242604 = True,
 ind_340812 = True,
 E_383726 = Ind!val!3222,
 ind_247998 = True,
 ind_332867 = True,
 ind_177499 = True,
 ind_77003 = True,
 ind_350801 = True,
 ind_74937 = True,
 ind_187399 = True,
 ind_288339 = True,
 ind_651 = True,
 ind_324581 = True,
 E_1052929 = Ind!val!30392,
 ind_251959 = True,
 ind_112529 = True,
 E_667307 = Ind!val!10399,
 ind_249979 = True,
 ind_23714 = True,
 ind_216254 = True,
 ind_315838 = True,
 E_123675 = Ind!val!5886,
 ind_140653 = True,
 ind_118912 = True,
 ind_157310 = True,
 E_168459 = Ind!val!9547,
 ind_185260 = True,
 ind_293837 = True,
 ind_161476 = True,
 ind_212038 = True,
 ind_302014 = True,
 ind_362339 = True,
 ind_117731 = True,
 ind_346744 = True,
 ind_76577 = True,
 E_762664 = Ind!val!44378,
 ind_200572 = True,
 ind_359356 = True,
 E_192325 = Ind!val!1247,
 ind_169418 = True,
 ind_192775 = True,
 ind_323249 = True,


# Loading PSL Ontology
This is a work in progress

In [ ]:
from opal.logic.z3.z3_ontology import Z3Ontology
psl_core_smt2 = ''
# Initialize the ontology
ontology = Z3Ontology.from_smt2(psl_core_smt2)

Adding axiom: ont_t_0_type_partition
Parsing expression: ForAll([x], Or(activity(x), activity_occurrence(x), object_(x), event(x), transition(x))
Parsing expression: Or(activity(x), activity_occurrence(x), object_(x), event(x), transition(x)
Parsing expression: activity(x)
Parsing expression: x
Parsing expression: activity_occurrence(x)
Parsing expression: x
Parsing expression: object_(x)
Parsing expression: x
Parsing expression: event(x)
Parsing expression: x
Parsing expression: transition(x
Parsing expression: x
Adding axiom: ont_t_1_type_disjointness
Parsing expression: ForAll([x], And(
    Implies(activity(x), Not(Or(activity_occurrence(x), object_(x), event(x), transition(x)))),
    Implies(activity_occurrence(x), Not(Or(object_(x), activity(x), event(x), transition(x)))),
    Implies(object_(x), Not(Or(activity_occurrence(x), activity(x), event(x), transition(x)))),
    Implies(event(x), Not(Or(activity_occurrence(x), activity(x), object_(x), transition(x)))),
    Implies(transit

Z3Exception: Sort mismatch

In [9]:
#s = Solver()
ref_taxonomy_path = '../../opal/logic/z3/ontologies/ref/reference_taxonomy.smt2'
ref_z3 = parse_smt2_file(ref_taxonomy_path)